In [1]:
import pandas as pd
import numpy as np
import sklearn
import torch.nn as nn
import sklearn.model_selection
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import pickle
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

In [2]:
def add_target(team):
    team['target'] = team['won'].shift(-1)
    return team

def rest_days(team):
    team['date_time'] = pd.to_datetime(team['date'])
    team['rest'] = (team['date_time'] - team['date_time'].shift(1)).dt.days
    team['rest'] = team['rest'].fillna(0)
    team['rest'] = team['rest'].astype(int)
    del team['date_time']
    return team

def winrate(team):
    total = team['Wins'] + team['Losses']
    total_opp = team['Wins_opp'] + team['Losses_opp']
    team['winrate'] = team['Wins'] / total
    team['winrate_opp'] = team['Wins_opp'] / total_opp
    return team

def differential(team):
    team['differential'] = team['Total'] - team['Total_opp']
    return team

def find_team_exp_average_5(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=5, adjust=False).mean()
    return rolling

def find_team_exp_average_9(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=9, adjust=False).mean()
    return rolling

def find_team_exp_average_12(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=12, adjust=False).mean()
    return rolling

def find_team_average_15(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(15).mean()
    return rolling

def find_team_average_10(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(10).mean()
    return rolling

def find_team_average_5(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(5).mean()
    return rolling

def find_team_average_3(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(3).mean()
    return rolling

def rolling(data):
    df_rolling_3 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_3 = df_rolling_3.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_3)
    df_rolling_5 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_5 = df_rolling_5.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_5)
    df_rolling_10 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_10 = df_rolling_10.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_10)
    df_rolling_15 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_15 = df_rolling_15.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_15)
    df_exp_rolling_5 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_5 = df_exp_rolling_5.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_5)
    df_exp_rolling_9 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_9 = df_exp_rolling_9.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_9)
    df_exp_rolling_12 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_12 = df_exp_rolling_12.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_12)
    exp_rolling_columns_5 = [f"{col}_exp_5" for col in df_exp_rolling_5.columns]
    exp_rolling_columns_9 = [f"{col}_exp_9" for col in df_exp_rolling_9.columns]
    exp_rolling_columns_12 = [f"{col}_exp_12" for col in df_exp_rolling_12.columns]
    rolling_columns_15 = [f"{col}_15" for col in df_rolling_15.columns]
    rolling_columns_10 = [f"{col}_10" for col in df_rolling_10.columns]
    rolling_columns_5 = [f"{col}_5" for col in df_rolling_5.columns]
    rolling_columns_3 = [f"{col}_3" for col in df_rolling_3.columns]
    df_exp_rolling_12.columns = exp_rolling_columns_12
    df_exp_rolling_9.columns = exp_rolling_columns_9
    df_exp_rolling_5.columns = exp_rolling_columns_5
    df_rolling_15.columns = rolling_columns_15
    df_rolling_10.columns = rolling_columns_10
    df_rolling_5.columns = rolling_columns_5
    df_rolling_3.columns = rolling_columns_3
    df = pd.concat([data, df_rolling_3, df_rolling_5, df_rolling_10, df_rolling_15, df_exp_rolling_5,df_exp_rolling_9, df_exp_rolling_12], axis=1)
    # df_exp_rolling_5,df_exp_rolling_9, df_exp_rolling_12
    return df

def ratio(feature):
    feature_opp = 'OPP_' + str(feature)
    free = nba[feature] / nba[feature_opp]
    return free

def ratios(nba):
    regard = []
    disregard = [col for col in nba.columns if "OPP_" in col]
    for col in disregard:
        col = col[4:100]
        if col in nba.columns:
            regard.append(col)
    nba_ratio = nba[regard].apply(ratio)
    nba_ratios_columns = [f"{col}_ratio" for col in nba_ratio.columns]
    nba_ratio.columns
    return regard

def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("Teams", group_keys=False).apply(lambda x: shift_col(x, col_name))

def date_change(datetime_str):
    # Parse the datetime string into a datetime object
    datetime_obj = datetime.strptime(datetime_str, '%m/%d/%Y')

    # Format the datetime object into a new string structure
    new_datetime_str = datetime_obj.strftime('%Y-%m-%d')

    return new_datetime_str

def haircut(df, date):
    df[date] = df[date].str[:10]
    return df

def convert_date_format(df):
    # Create a boolean mask to identify values in the "m/d/y" format
    mask = df['Date'].str.contains(r'\d{1,2}/\d{1,2}/\d{2}')
    
    # Apply the conversion only to values that match the mask
    df.loc[mask, 'Date'] = nba.loc[mask, 'Date'].apply(date_change)
    return df

def spread(df):
    # construct spread between home and away rankings
    ranks = [i for i in df.columns if 'RANK' in i]
    ranks_home = [i for i in ranks if '_x' in i]
    ranks_away = [i for i in ranks if '_y' in i]
    spread_columns_names = [col[:-1] for col in ranks_home]
    
    spread_columns = []
    for base_name in spread_columns_names:
        spread_column = df[f'{base_name}x'] - df[f'{base_name}y']
        spread_column.name = f'{base_name}spread'
        spread_columns.append(spread_column)
        
    spread_df = pd.concat(spread_columns, axis=1)
    return spread_df

def last_season(df):
    # construct winrate from last season
    df['last_season_winrate'] = df.groupby('Teams', 'season')['winrate'].last()

In [3]:
folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/raw_data/NBA_2018_2024.csv"
#folder_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/data/raw_data/NBA_2018_2024.csv"

df = pd.read_csv(folder_path, index_col=0)

folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/nba_api/data/teams_stats/processed_cum_2018_2024.csv"
#folder_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/processed_cumulative_season_stats_2019_2024.csv"
nba = pd.read_csv(folder_path, index_col=0)

folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/odds_data/2021_01_20_onward.csv"
#folder_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/data/odds_data/2021_01_20_onward.csv"
odds = pd.read_csv(folder_path, index_col=0)

In [4]:
nba[nba['Teams'] == 'PHO'].iloc[-1]['Date']

'2024-04-03'

In [9]:
# nba dataframe does not include the 2018 season
df = df[~df['season'].isin([2018])]
df = df.reset_index(drop=True)
df = haircut(df, 'date')

# rename nba columns to match df
nba = haircut(nba, 'Date')
nba = convert_date_format(nba)
nba.rename(columns={'Date': 'date'}, inplace=True)

#rename odds columns to match df
odds.rename(columns={'Timestamp': 'date_next'}, inplace=True)

# construct winrate for team
df = winrate(df)
# construct differential points
df = differential(df)
# construct target
df = df.groupby("Teams", group_keys=False).apply(add_target)
# construct resting
df = df.groupby(["Teams",'season'], group_keys=False).apply(rest_days)
# games yet to play are 2
df.loc[pd.isnull(df['target']), 'target'] = 2
# convert win/loss to 1/0
df['target'] = df['target'].astype(int)

# remove metadata and target for df
removed = ['target', 'date', 'Teams_opp', 'Teams',
           'season','won', 'Wins', 'Losses', 
           'Wins_opp', 'Losses_opp']
valid_columns = df.columns[~df.columns.isin(removed)]

# scale the data for df
scaler = MinMaxScaler()
df[valid_columns] = scaler.fit_transform(df[valid_columns])

# construct rolling features to df
df = rolling(df).copy()
df = df.dropna()

# remove metadata for nba ranking 
removed = ['date', 'Teams']
valid_columns = nba.columns[~nba.columns.isin(removed)]

# scale the ranking data
scaler = MinMaxScaler()
nba[valid_columns] = scaler.fit_transform(nba[valid_columns])

# remove metadata for odds data
# removed = list(odds.columns[odds.dtypes == 'object'])
# valid_columns = odds.columns[~nba.columns.isin(removed)]

# scale the odds data
# scaler = MinMaxScaler()
# odds[valid_columns] = scaler.fit_transform(odds[valid_columns])

# construct current game metadata for df
df['home_next'] = add_col(df, 'home')
df['team_next_opp'] = add_col(df, 'Teams_opp')
df['date_next'] = add_col(df, 'date')
df = df.copy()

full = pd.merge(df, nba, on=['Teams', 'date'], how='left') # problem here 

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_13229/1808437805.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("Teams", group_keys=False).apply(add_target)
/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_13229/1808437805.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(["Teams",'season'], group_keys=False).apply(rest_days)
/var/folders/99/zv3w

In [10]:
full[full['Teams'] == 'PHO'].iloc[-1]['date']

'2024-04-03'

## Construction of Future Games
We need to now construct a subset of the dataframe to reflect future games.

In [11]:
mapping = {'Brooklyn Nets': 'BRK', 'Golden State Warriors': 'GSW', 'Los Angeles Lakers': 'LAL',
       'Milwaukee Bucks': 'MIL', 'Boston Celtics': 'BOS', 'Charlotte Hornets': 'CHO',
       'Chicago Bulls': 'CHI', 'Cleveland Cavaliers': 'CLE', 'Denver Nuggets': 'DEN',
       'Detroit Pistons': 'DET', 'Houston Rockets': 'HOU', 'Indiana Pacers': 'IND',
       'Memphis Grizzlies': 'MEM', 'Minnesota Timberwolves': 'MIN',
       'New Orleans Pelicans': 'NOP', 'New York Knicks': 'NYK', 'Oklahoma City Thunder': 'OKC',
       'Orlando Magic': 'ORL', 'Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHO',
       'Portland Trail Blazers': 'POR', 'Sacramento Kings': 'SAC', 'San Antonio Spurs': 'SAS',
       'Toronto Raptors': 'TOR', 'Utah Jazz': 'UTA', 'Washington Wizards': 'WAS',
       'Atlanta Hawks': 'ATL', 'Dallas Mavericks': 'DAL', 'LA Clippers': 'LAC', 'Miami Heat': 'MIA'}

In [12]:
# Need to update this 
future_teams = ['PHI', 'MIA', 'SAC', 'NYK', 'ATL', 'DAL', 'GSW', 'HOU', 'DEN', 'LAC']

In [13]:
def construction(df, teams):
    constructed = []
    # teams are the teams that will play in the future
    for team in teams:
        constructed.append(df[df['Teams'] == team].iloc[-1])
    tonight = pd.concat(constructed, axis=1).T
    tonight = tonight.reset_index(drop=True)
    return tonight

def opponent(tonight):
    figure = []
    for i in range(len(tonight['Teams'])):
        if (i % 2 == 0):
            figure.append(tonight['Teams'][i+1])
        else:
            figure.append(tonight['Teams'][i-1])
    return figure

In [14]:
tonight = construction(full, future_teams)

In [15]:
tonight['home_next'] = [0,1] * 5 # need to update this
tonight['team_next_opp'] = opponent(tonight)
date_to_repeat = '2024-04-04' # need to update this
tonight['date_next'] = [date_to_repeat] * len(tonight)

In [16]:
# merge stats from opposing teams
complete = tonight.merge(tonight,
               left_on=['Teams', 'date_next'],
               right_on = ['team_next_opp', 'date_next'])

# concat the spreads
spread_df = spread(complete)
complete = pd.concat([complete, spread_df], axis=1)

# concat the odds
complete = pd.merge(complete, odds, on=['Teams_x', 'date_next', 'Teams_y'], how='left')
complete = complete.dropna()

In [17]:
complete

mp_x      fg_x     fga_x     fg%_x      3p_x     3pa_x     3p%_x      ft_x  \
0  0.0  0.340909      0.25  0.461722  0.444444  0.333333  0.514877       0.5   
1  0.0  0.340909  0.183333  0.519139  0.555556  0.466667  0.489004  0.428571   
2  0.0  0.409091  0.483333  0.370813   0.37037  0.533333  0.280724  0.404762   
3  0.0  0.318182  0.216667   0.45933   0.37037       0.4    0.3674  0.357143   
4  0.0  0.636364      0.45  0.657895  0.407407  0.466667  0.353169  0.190476   
5  0.0  0.409091  0.433333  0.404306  0.481481       0.5   0.39586  0.119048   
6  0.0  0.454545      0.45  0.447368  0.481481  0.366667  0.517464  0.119048   
7  0.0  0.454545  0.483333  0.423445  0.444444  0.633333  0.288486  0.190476   
8  0.0       0.5  0.583333  0.406699  0.333333       0.4  0.329884  0.261905   
9  0.0  0.227273  0.216667  0.332536  0.333333  0.283333  0.437257   0.47619   

      fta_x     ft%_x     orb_x     drb_x     trb_x     ast_x     stl_x blk_x  \
0  0.431373  0.891452  0.310345  0.386364  0.384615  0.452381  0.363636   0.2   
1  0.352941  0.934871  0.241379  0.363636  0.326923  0.380952  0.227273   0.2   
2  0.431373  0.674355  0.689655  0.454545  0.653846       0.5  0.454545   0.3   
3  0.352941  0.742198  0.241379  0.159091  0.153846  0.404762       0.5   0.1   
4  0.235294  0.548168  0.206897  0.590909       0.5  0.666667  0.409091   0.4   
5  0.156863  0.506106  0.275862  0.409091  0.384615  0.452381  0.363636  0.35   
6  0.078431       1.0   0.37931  0.568182  0.576923       0.5  0.409091   0.2   
7  0.215686   0.61194  0.413793  0.204545  0.288462  0.357143  0.454545   0.0   
8  0.254902  0.746269   0.37931  0.568182  0.576923   0.47619  0.227273   0.5   
9  0.470588  0.748982  0.310345  0.295455  0.307692  0.238095  0.272727  0.05   

      tov_x     pf_x     pts_x     ts%_x    efg%_x    3par_x     ftr_x  \
0  0.517241   0.4375   0.37963  0.566372  0.506073  0.473413  0.407932   
1  0.551724  0.34375   0.37963  0.661504  0.607287  0.694683  0.355524   
2  0.310345    0.375   0.37963  0.382743  0.380567  0.596913  0.338527   
3  0.448276  0.34375  0.287037  0.497788  0.481781   0.57976  0.344193   
4  0.517241  0.21875  0.490741  0.597345  0.639676  0.536878  0.189802   
5  0.482759     0.25  0.296296  0.391593  0.447368   0.58319  0.128895   
6  0.551724    0.375  0.333333  0.449115  0.481781  0.423671  0.063739   
7  0.448276  0.53125  0.351852   0.40708  0.447368  0.708405  0.168555   
8  0.241379  0.28125  0.388889  0.371681  0.392713  0.408233  0.184136   
9  0.482759  0.46875      0.25  0.409292  0.362348  0.421955  0.456091   

     orb%_x    drb%_x    trb%_x    ast%_x    stl%_x    blk%_x    tov%_x  \
0  0.403226  0.521505  0.483425  0.739599  0.392523  0.205446  0.529851   
1  0.417563  0.679211  0.679558   0.61171  0.257009  0.235149  0.597015   
2  0.731183  0.641577  0.765193  0.730354  0.514019   0.30198  0.291045   
3  0.320789  0.582437  0.320442  0.684129  0.570093  0.138614  0.488806   
4  0.306452  0.713262  0.679558  0.716487  0.401869  0.309406  0.492537   
5  0.292115  0.562724  0.353591  0.650231  0.378505  0.299505  0.477612   
6  0.437276  0.707885  0.646409  0.673344  0.429907   0.20297  0.537313   
7  0.478495   0.65233  0.464088  0.448382  0.485981       0.0  0.429104   
8  0.358423  0.519713  0.411602  0.587057  0.233645  0.405941  0.227612   
9  0.358423  0.268817  0.234807  0.479199  0.308411  0.049505  0.507463   

  usg%_x    ortg_x    drtg_x fgmaxes_x fgamaxes_x fg%maxes_x 3pmaxes_x  \
0    0.0  0.513216   0.46696  0.227273   0.162162        1.0  0.307692   
1    0.0  0.575991  0.453744  0.318182   0.135135      0.445  0.538462   
2    0.0  0.571586  0.401982  0.227273    0.27027        1.0  0.230769   
3    0.0  0.453744  0.575991  0.363636   0.297297        1.0  0.384615   
4    0.0  0.526432   0.44163  0.363636   0.297297        1.0  0.153846   
5    0.0  0.365639  0.410793  0.363636   0.324324      0.375  0.307692   
6    0.0  0.410793  0.365639  0.227273   0.216216      0.445  0.230769

In [18]:
# load models and features
save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/ml_notebooks/factors/'
file_name = 'calibrated_predictors_ridge_classifier_30_predictors_19.97%_2019_2024.txt'
file_path = save_path + file_name
with open(file_path, 'r') as f:
    predictors = f.read()
    
# model
save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/ml_notebooks/weights/'
file_name = 'calibrated_ridge_classifier_30_predictors_19.97%_2019_2024.pkl'
file_path = save_path + file_name
with open(file_path, 'rb') as f:
    clf = pickle.load(f)

predictors = predictors.split(',')[:-1]
X = complete[predictors]
predictions = clf.predict(X)
result = pd.Series(predictions, index = X.index)
final = pd.concat([complete['Teams_x'], result], axis=1)

In [19]:
predictors

['usg%_x',
 '+/-maxes_x',
 'winrate_opp_x',
 'orbmaxes_3_x',
 'blkmaxes_3_x',
 'ts%_opp_3_x',
 'stl%maxes_opp_3_x',
 'trb%_5_x',
 'trb%_opp_5_x',
 'stl%maxes_opp_5_x',
 'drtgmaxes_opp_5_x',
 'astmaxes_opp_15_x',
 'ft%maxes_exp_5_x',
 'stl%maxes_opp_exp_9_x',
 'winrate_opp_exp_9_x',
 'fg_exp_12_x',
 'trb_exp_12_x',
 'ts%maxes_exp_12_x',
 'FGA_base_x',
 'FGA_RANK_base_x',
 'drb%maxes_y',
 '3pamaxes_3_y',
 'stl%maxes_opp_3_y',
 'fg%_5_y',
 'efg%_exp_5_y',
 'blk%maxes_exp_12_y',
 'FGA_RANK_base_y',
 'FGA_RANK_base_spread',
 'Draftkings_odds_x',
 'Fanduel_odds_y']

In [20]:
probabilities = clf.predict_proba(X)[:, 1]
result = pd.DataFrame({
    'Predictions': predictions,
    'Probabilities': probabilities,
    'Fanduel': complete['Fanduel_odds_x'],
    'Draftkings': complete['Draftkings_odds_x'],
    'BetMGM': complete['BetMGM_odds_x']
}, index=X.index)

# Concatenate the predictions and probabilities with the original data
final = pd.concat([complete['date_next'],complete['Teams_x'], result], axis=1)
final.columns = ['Date','Teams', 'Predictions', 'Probabilities', 'Fanduel', 'Draftkings', 'BetMGM']
# final.loc[final['Teams'] == 'POR', 'BetMGM'] = 1.90
# final.loc[final['Teams'] == 'POR', 'Draftkings'] = 1.90
# final.loc[final['Teams'] == 'POR', 'Fanduel'] = 1.90
final['Our_Odds'] = 1 / final['Probabilities']
final['Best_Odds'] = final[['Fanduel', 'Draftkings', 'BetMGM']].max(axis=1)

In [21]:
from sklearn.metrics import accuracy_score
# final['Actual'] = [0,1,1,0,0,1,0,1,0,1,0,1,1,0,0,1,]
# accuracy = accuracy_score(final['Actual'], final['Predictions'])
# accuracy

In [22]:
final

,Date,Teams,Predictions,Probabilities,Fanduel,Draftkings,BetMGM,Our_Odds,Best_Odds
0,2024-04-04,PHI,0,0.426471,2.10,2.10,2.10,2.344828,2.10
1,2024-04-04,MIA,1,0.581818,1.77,1.77,1.74,1.718750,1.77
2,2024-04-04,SAC,0,0.395455,2.30,2.30,2.35,2.528736,2.35
3,2024-04-04,NYK,1,0.581818,1.65,1.65,1.62,1.718750,1.65
4,2024-04-04,ATL,0,0.180000,6.00,5.90,6.00,5.555556,6.00
5,2024-04-04,DAL,1,0.785714,1.14,1.15,1.14,1.272727,1.15
6,2024-04-04,GSW,1,0.581818,1.60,1.59,1.61,1.718750,1.61
7,2024-04-04,HOU,0,0.395455,2.40,2.42,2.40,2.528736,2.42
8,2024-04-04,DEN,1,0.620690,1.60,1.57,1.57,1.611111,1.60
9,2024-04-04,LAC,0,0.395455,2.42,2.45,2.45,2.528736,2.45


In [23]:
win = final[final['Predictions'] == 1]
bet = win[win['Our_Odds'] < win['Best_Odds']]
bet['odds_diff'] = bet['Best_Odds'] - bet['Our_Odds']

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_13229/3053637661.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bet['odds_diff'] = bet['Best_Odds'] - bet['Our_Odds']


In [28]:
bet

,Date,Teams,Predictions,Probabilities,Fanduel,Draftkings,BetMGM,Our_Odds,Best_Odds,odds_diff,Kelly
1,2024-04-04,MIA,1,0.581818,1.77,1.77,1.74,1.71875,1.77,0.05125,0.019362


In [24]:
def kelly(decimal_odds, P, fractional):
    # shrinkage is the percent we want to shrink bets to account for uncertanity
    B = decimal_odds - 1
    Q = 1 - P
    solution = (B*P - Q) / B
    solution = solution * fractional
    return solution

In [25]:
bet['Kelly'] = kelly(bet['Best_Odds'], bet['Probabilities'], 0.50)

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_13229/2290681703.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bet['Kelly'] = kelly(bet['Best_Odds'], bet['Probabilities'], 0.50)


In [26]:
bet_very_undervalued = bet[bet['odds_diff'] > 0.06]
bet_very_undervalued = bet_very_undervalued[bet_very_undervalued['Best_Odds'] > 1.1]

In [27]:
bet_very_undervalued

,Date,Teams,Predictions,Probabilities,Fanduel,Draftkings,BetMGM,Our_Odds,Best_Odds,odds_diff,Kelly
